---
title: Python 문법 연습 심화(토이 프로그램)
---

> Sudoku Solver와 Page Ranker 등을 구현하면서 Python 문법을 연습합니다.

## Sudoku Solver(Pure Python)

In [1]:

import platform
import os

CLS = "cls" if platform.system() == "Windows" else "clear"

def make_board(problem):
    sudoku = []
    row = []
    for i in range(len(problem)):
        row.append(int(problem[i]))
        if (i+1) % 9 == 0:
            sudoku.append(row)
            row = []
    return sudoku
    
def print_sudoku(board):
    for i in range(len(board)):
        if i % 3 == 0 and i != 0:
            print("------+-------+-------")
        for j in range(len(board[0])):
            if j % 3 == 0 and j != 0:
                print("| ", end="")
            if j == 8:
                if board[i][j] == 0:
                    print(" ")
                else:
                    print(board[i][j])
            else:
                if board[i][j] == 0:
                    print(" " + " ", end="")
                else:
                    print(str(board[i][j]) + " ", end="")

def is_empty(board):
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 0:
                return (i, j)
    return None

def is_valid(board, num, pos):
    for i in range(len(board[0])):
        if board[pos[0]][i] == num and pos[1] != i:
            return False
    for i in range(len(board)):
        if board[i][pos[1]] == num and pos[0] != i:
            return False
    boardx_x = pos[1] // 3
    boardx_y = pos[0] // 3
    for i in range(boardx_y*3, boardx_y*3 + 3):
        for j in range(boardx_x * 3, boardx_x*3 + 3):
            if board[i][j] == num and (i,j) != pos:
                return False
    return True

def solve(board):
    find = is_empty(board)
    if not find:
        return True
    else:
        row, col = find
    for i in range(1,10):
        if is_valid(board, i, (row, col)):
            board[row][col] = i
            os.system(CLS)
            print(f"\nSolution {row, col}: \n")
            print_sudoku(board)
            if solve(board):
                return True
            board[row][col] = 0
    return False

if __name__ == "__main__":
    problem = f"405001068073628500009003070240790030006102005950000021507064213080217050612300007"
    board = make_board(problem)
    print("Problem:")
    print_sudoku(board)
    solve(board)

Problem:
4   5 |     1 |   6 8
  7 3 | 6 2 8 | 5    
    9 |     3 |   7  
------+-------+-------
2 4   | 7 9   |   3  
    6 | 1   2 |     5
9 5   |       |   2 1
------+-------+-------
5   7 |   6 4 | 2 1 3
  8   | 2 1 7 |   5  
6 1 2 | 3     |     7

Solution (0, 1): 

4 2 5 |     1 |   6 8
  7 3 | 6 2 8 | 5    
    9 |     3 |   7  
------+-------+-------
2 4   | 7 9   |   3  
    6 | 1   2 |     5
9 5   |       |   2 1
------+-------+-------
5   7 |   6 4 | 2 1 3
  8   | 2 1 7 |   5  
6 1 2 | 3     |     7

Solution (0, 3): 

4 2 5 | 9   1 |   6 8
  7 3 | 6 2 8 | 5    
    9 |     3 |   7  
------+-------+-------
2 4   | 7 9   |   3  
    6 | 1   2 |     5
9 5   |       |   2 1
------+-------+-------
5   7 |   6 4 | 2 1 3
  8   | 2 1 7 |   5  
6 1 2 | 3     |     7

Solution (0, 4): 

4 2 5 | 9 7 1 |   6 8
  7 3 | 6 2 8 | 5    
    9 |     3 |   7  
------+-------+-------
2 4   | 7 9   |   3  
    6 | 1   2 |     5
9 5   |       |   2 1
------+-------+-------
5   7 |   6 4 | 2 1 3

## Sudoku Solver(Numpy)

In [2]:
import numpy as np

def make_board(problem):
    return np.array([int(i) for i in problem]).reshape(9,9)

def print_sudoku(problem):
    # 인지값 확인
    assert problem.shape == (9, 9)
    assert type(problem) == np.ndarray
    
    # 출력의 편의성을 위해서 문자열로 변경
    board_str = problem.astype(str)
    
    row_sep = '-'*25
    for i in range(9):        
        if i % 3 == 0:
            print(row_sep)
        row = board_str[i]
        print('| '+' '.join(row[0:3])+' | '+' '.join(row[3:6])+' | '+' '.join(row[6:])+' |')
    print(row_sep)

def roundup_to_nearest_three(index):
    roundup_float = np.ceil((index + 1) / 3) * 3 # Add 1 as indices start from 0
    roundup_int = int(roundup_float)    
    return roundup_int

def check_unique(board, row, column):
    # Get distinct values from row and column
    row_values = np.unique(board[row,:])
    col_values = np.unique(board[:,column])
    
    # First define the sub cell that the row/column falls into
    # - This will be a group of 3 in each axis
    row_end_pos = roundup_to_nearest_three(row)
    col_end_pos = roundup_to_nearest_three(column)
    
    # - Then get distinct values from sub cells
    box_values = np.unique(board[row_end_pos-3:row_end_pos, 
                                 col_end_pos-3:col_end_pos])
    
    # - Bring all into one list
    all_values = np.concatenate((row_values, col_values, box_values), axis=None)
    
    # - Then take the unique values from all of them
    unique_values = np.unique(all_values)
    
    return unique_values

def fill_values(board, row, column):
    # We're only interested in values not yet filled
    if board[row,column] == 0:
        existing_values = check_unique(board, row, column)
        potential_values = [value for value in range(1,10) if value not in existing_values]
        
        # If there's only one potential solution, overwrite zero with that value
        if len(potential_values) == 1:
            board[row,column] = potential_values[0]
            # print('Row: ', str(row + 1), '& Col: ', str(column + 1), ' overwritten with ', str(potential_values[0]))

def solve(board):
    # Restrict to max of 10 loops of the board
    for i in range(10):    
        # Loop through table columns & rows
        for row in range(9):
            for column in range(9):
                fill_values(board, row, column)
                    
        # print('\n Loop number ', str(i + 1), ' complete \n')
        
        # Checks array for number of non-filled values remaining
        zeroes_remaining = np.count_nonzero(board == 0)
        
        if zeroes_remaining == 0:
            # print('Finished!')
            break
        else:
            pass
            # print(' ', str(zeroes_remaining), ' zeroes left\n')

if __name__ == "__main__":
    problem = "405001068073628500009003070240790030006102005950000021507064213080217050612300007"
    board = make_board(problem)
    print("Problem:")
    print_sudoku(board)
    print("Solve:")
    solve(board)
    print_sudoku(board)

Problem:
-------------------------
| 4 0 5 | 0 0 1 | 0 6 8 |
| 0 7 3 | 6 2 8 | 5 0 0 |
| 0 0 9 | 0 0 3 | 0 7 0 |
-------------------------
| 2 4 0 | 7 9 0 | 0 3 0 |
| 0 0 6 | 1 0 2 | 0 0 5 |
| 9 5 0 | 0 0 0 | 0 2 1 |
-------------------------
| 5 0 7 | 0 6 4 | 2 1 3 |
| 0 8 0 | 2 1 7 | 0 5 0 |
| 6 1 2 | 3 0 0 | 0 0 7 |
-------------------------
Solve:
-------------------------
| 4 2 5 | 9 7 1 | 3 6 8 |
| 1 7 3 | 6 2 8 | 5 9 4 |
| 8 6 9 | 5 4 3 | 1 7 2 |
-------------------------
| 2 4 1 | 7 9 5 | 8 3 6 |
| 7 3 6 | 1 8 2 | 9 4 5 |
| 9 5 8 | 4 3 6 | 7 2 1 |
-------------------------
| 5 9 7 | 8 6 4 | 2 1 3 |
| 3 8 4 | 2 1 7 | 6 5 9 |
| 6 1 2 | 3 5 9 | 4 8 7 |
-------------------------


## Sudoku Solver(Numpy를 사용해서 빠르게 부분해를 찾는 방법)
> 선형대수를 이용하여 Sudoku Solver를 구현

![Cube](https://miro.medium.com/v2/resize:fit:1280/format:webp/1*Q0dCE_AY_Y4d65G97wec1A.png)
![Diagonal](https://miro.medium.com/v2/resize:fit:1280/format:webp/1*FcsHVRKHJUelYKL0P7e3Mg.png)

In [3]:
import numpy as np
from numpy.lib.stride_tricks import as_strided

problem = "405001068073628500009003070240790030006102005950000021507064213080217050612300007"
board = np.array([int(i) for i in problem]).reshape(9,9)
print(board.strides)

def as_subsquares(board):
    S =board.itemsize
    ast = as_strided(board, shape=(3,3,3,3), strides=(S*27, S*3, S*9, S)).reshape(9,3,3)
    return ast.copy()

def as_board(subsqrs):
    a = subsqrs.reshape(3,3,3,3)
    a = np.hstack(a)
    b = np.hstack(a)
    return b.copy()

def as_subcuboids(cube):
    S = cube.itemsize
    cuboids = as_strided(cube, shape=(3,3,3,3,3,3), strides=( S*27, S*3, S*81*3, S*81, S*9, S)).reshape(9,9,3,3)
    return cuboids.copy()

def create_validation_cube():
    corner_rod = np.arange(1,10)
    c2d = corner_rod[:,None]*np.ones((9,9))
    c3d = c2d[:,:,None]*np.ones((9,9,9))
    return c3d

def eliminate_cube(cube, board, pts):
    x,y = pts
    elm = board[x, y]
    cube[elm-1,x, :] = 0
    cube[elm-1, :, y] = 0
    cube[:, x, y] = 0
    start_x, start_y= x//3*3,  y//3*3
    end_x, end_y = start_x + 3, start_y + 3
    cube[elm-1, start_x:end_x, start_y:end_y] = 0

def validate_insertion(elm, pt, cube, board):
    x,y = pt
    bkp_brd_elm = board[x,y]
    board[x,y] = elm
    bkp_vcube_plane = cube[elm-1, ...]
    cube[elm-1, x, :] = 0
    cube[elm-1, :, y] = 0
    bkp_vcube_xy = cube[:, x, y]
    cube[:, x, y] = 0
    start_x, start_y= x//3*3,  y//3*3
    end_x, end_y = start_x + 3, start_y + 3
    cube[elm-1, start_x:end_x, start_y:end_y] = 0
    cuboids = as_subcuboids(cube)
    bsqrs = as_subsquares(board)
    
    valid_insertion = True
    for bblck, cuboid in zip(bsqrs, cuboids):
        nonzero_elms = bblck[bblck!=0]
        absent_elms = np.setdiff1d(np.arange(9)+1, nonzero_elms)        
        count_present_elms_from_board_block = len(nonzero_elms)
        count_present_elms_from_cuboid = (~np.any(cuboid[nonzero_elms-1,...], axis=(1,2))).sum()
        if count_present_elms_from_cuboid != count_present_elms_from_board_block:
            valid_insertion = False
            break
        count_absent_elms_from_cuboid = np.any(cuboid[absent_elms-1,...], axis=(1,2)).sum()
        count_absent_elms_from_board_block = np.count_nonzero(bblck==0)
        if count_absent_elms_from_cuboid != count_absent_elms_from_board_block:
            valid_insertion = False
            break
    cube[:,x,y] = bkp_vcube_xy
    cube[elm-1,...] = bkp_vcube_plane
    board[x,y] = bkp_brd_elm
    return valid_insertion

print(board)
print("Start")
prev_board = board.copy()
repeat = 0
step = 1

non_zero_indices = np.transpose(prev_board.nonzero())
vcube = create_validation_cube()

while True:
    print(f"Step - {step}")
    for pt in non_zero_indices:
        eliminate_cube(vcube, prev_board, pt)
    cuboids = as_subcuboids(vcube)
    subsqrs = as_subsquares(prev_board)
    new_non_zero_pts = []
    for sqr_idx, zipped in enumerate(zip(cuboids, subsqrs)):
        cuboid, sb_sqr = zipped
        mask_count1 = (np.bincount(cuboid.nonzero()[0], minlength=9)==1)
        planes_with_1_elm = cuboid[mask_count1,...]
        plane_with_1_elm = np.sum(planes_with_1_elm, axis=0)
        zero_count = np.count_nonzero(plane_with_1_elm==0)
        if (plane_with_1_elm.size == 0) or (zero_count==9):
            continue
        elm_plane_x, elm_plane_y = plane_with_1_elm.nonzero()
        sb_sqr[elm_plane_x, elm_plane_y] = plane_with_1_elm[elm_plane_x, elm_plane_y]
        brd_x = sqr_idx//3*3 + elm_plane_x
        brd_y = sqr_idx%3*3  + elm_plane_y
        new_non_zero_pts.append(np.transpose((brd_x, brd_y)))
    solved_board = as_board(subsqrs)
    print(solved_board)
    if len(new_non_zero_pts)>1:
       new_non_zero_pts = np.vstack(new_non_zero_pts)
       non_zero_indices = new_non_zero_pts
    count_zero = np.count_nonzero(solved_board==0)
    if count_zero == 0:
        break
    elif np.array_equal(solved_board, prev_board):
        repeat += 1
    else:
        print(f"Places left to be filled : {count_zero}")
        prev_board = solved_board
    if repeat != 0:
        print("Board repeated. Partial solution acheived.")
        print("Need backtracking solution.")
        break
    step += 1

(36, 4)
[[4 0 5 0 0 1 0 6 8]
 [0 7 3 6 2 8 5 0 0]
 [0 0 9 0 0 3 0 7 0]
 [2 4 0 7 9 0 0 3 0]
 [0 0 6 1 0 2 0 0 5]
 [9 5 0 0 0 0 0 2 1]
 [5 0 7 0 6 4 2 1 3]
 [0 8 0 2 1 7 0 5 0]
 [6 1 2 3 0 0 0 0 7]]
Start
Step - 1
[[4 0 5 9 7 1 3 6 8]
 [0 7 3 6 2 8 5 0 0]
 [8 6 9 0 0 3 1 7 2]
 [2 4 1 7 9 5 0 3 0]
 [7 0 6 1 0 2 0 0 5]
 [9 5 0 0 0 0 0 2 1]
 [5 9 7 0 6 4 2 1 3]
 [3 8 4 2 1 7 0 5 0]
 [6 1 2 3 0 0 0 0 7]]
Places left to be filled : 24
Step - 2
[[4 2 5 9 7 1 3 6 8]
 [1 7 3 6 2 8 5 0 0]
 [8 6 9 0 0 3 1 7 2]
 [2 4 1 7 9 5 0 3 0]
 [7 3 6 1 0 2 0 0 5]
 [9 5 8 0 0 6 7 2 1]
 [5 9 7 0 6 4 2 1 3]
 [3 8 4 2 1 7 0 5 0]
 [6 1 2 3 5 9 0 0 7]]
Places left to be filled : 16
Step - 3
[[4 2 5 9 7 1 3 6 8]
 [1 7 3 6 2 8 5 0 0]
 [8 6 9 5 0 3 1 7 2]
 [2 4 1 7 9 5 0 3 0]
 [7 3 6 1 8 2 0 0 5]
 [9 5 8 0 3 6 7 2 1]
 [5 9 7 8 6 4 2 1 3]
 [3 8 4 2 1 7 0 5 0]
 [6 1 2 3 5 9 0 0 7]]
Places left to be filled : 12
Step - 4
[[4 2 5 9 7 1 3 6 8]
 [1 7 3 6 2 8 5 0 0]
 [8 6 9 5 4 3 1 7 2]
 [2 4 1 7 9 5 8 3 0]
 [7 3 6 1 8 2 0 